# Day 5 - 改代码：修改OpenClaw的一个简单功能

> "动手实践：用模型思维修改OpenClaw源码，提交你的第一个PR"

---

## 课前回顾

前四天我们学习了：
- Day 1：体验维护地狱
- Day 2：理解模型思维
- Day 3：掌握MECE原则
- Day 4：Prompt 2.0与AI协作

今天，我们将**真正动手**，修改OpenClaw源码！

## 1. 环境准备

### 1.1 Clone OpenClaw代码

```bash
# 1. Fork OpenClaw仓库到你自己的GitHub账号
# 访问 https://github.com/openclaw/openclaw 点击 Fork

# 2. Clone你的Fork
git clone https://github.com/YOUR_USERNAME/openclaw.git
cd openclaw

# 3. 添加上游仓库
git remote add upstream https://github.com/openclaw/openclaw.git

# 4. 创建新分支
git checkout -b feature/my-first-change
```

### 1.2 安装依赖

```bash
# 使用pnpm（推荐）
pnpm install

# 或使用npm
npm install
```

### 1.3 验证环境

```bash
# 运行测试
pnpm test

# 构建
pnpm build
```

如果以上步骤都成功，说明环境配置完成！

## 2. 用模型思维阅读代码

### 2.1 OpenClaw目录结构

```
openclaw/
├── src/
│   ├── commands/          # CLI命令实现
│   ├── telegram/          # Telegram通道
│   ├── discord/           # Discord通道
│   ├── slack/             # Slack通道
│   ├── routing/           # 消息路由
│   ├── agents/            # AI代理
│   ├── channels/          # 通道抽象
│   └── infra/             # 基础设施
├── extensions/            # 扩展插件
├── docs/                  # 文档
└── test/                  # 测试
```

**模型对应关系：**
- `channels/` → 通道上下文
- `routing/` → 路由上下文
- `agents/` → 代理上下文

### 2.2 找到入口点

```
消息处理流程（从Telegram为例）：

src/telegram/
├── telegram-provider.ts   # 入口：接收消息
├── telegram-formatter.ts  # 格式化响应
└── ...

src/routing/
├── router.ts              # 路由核心
├── message-handler.ts     # 消息处理
└── ...

src/agents/
├── agent.ts               # 代理定义
├── agent-runner.ts        # 代理执行
└── ...
```

### 2.3 阅读代码的技巧

**技巧1：先读测试**

```typescript
// 测试文件告诉你这个模块做什么
// test/routing/router.test.ts
describe('Router', () => {
  it('should route message to correct handler', () => {
    // 了解Router的预期行为
  });
});
```

**技巧2：画调用图**

```
TelegramProvider.receive()
        ↓
Router.route()
        ↓
MessageHandler.handle()
        ↓
AgentRunner.run()
        ↓
TelegramProvider.send()
```

**技巧3：关注接口而非实现**

```typescript
// 先读接口定义
interface MessageHandler {
  handle(message: Message): Promise<Response>;
}

// 理解职责后，再看具体实现
class DefaultMessageHandler implements MessageHandler {
  // 具体实现...
}
```

## 3. 任务：添加日志功能

### 3.1 任务描述

在消息处理的关键节点添加日志输出，帮助调试和理解流程。

### 3.2 定位修改点

根据模型思维，日志应该在哪里添加？

```
接入层：接收消息时 → "收到消息，来自：{channel}"
处理层：路由决策时 → "路由决策：{messageType} → {handler}"
执行层：调用AI时   → "调用代理：{agent}"
输出层：发送响应时 → "发送响应，长度：{length}"
```

### 3.3 找到具体文件

```bash
# 搜索关键词
grep -r "receive" src/telegram/ --include="*.ts"
grep -r "route" src/routing/ --include="*.ts"
grep -r "handle" src/routing/ --include="*.ts"
```

### 3.4 实施修改

```typescript
// src/routing/router.ts

// 添加日志导入
import { logger } from '../infra/logger';

class Router {
  async route(message: Message): Promise<Handler> {
    // 添加日志
    logger.info(`Routing message: type=${message.type}, user=${message.userId}`);
    
    const handler = this.findHandler(message);
    
    logger.info(`Selected handler: ${handler.name}`);
    
    return handler;
  }
}
```

## 4. 测试你的修改

### 4.1 运行相关测试

```bash
# 运行路由相关的测试
pnpm test routing

# 运行所有测试
pnpm test
```

### 4.2 手动验证

```bash
# 启动OpenClaw（本地模式）
pnpm dev

# 在另一个终端发送测试消息
pnpm openclaw message send --text "Hello" --channel test

# 观察日志输出
```

### 4.3 检查代码规范

```bash
# 运行lint
pnpm lint

# 自动修复
pnpm lint:fix

# 格式化
pnpm format
```

## 5. 提交你的PR

### 5.1 提交代码

```bash
# 查看修改
git status
git diff

# 添加修改
git add .

# 提交（遵循提交规范）
git commit -m "feat(routing): add logging to message routing

- Log message type and user on receive
- Log selected handler on route
- Helps debugging and monitoring

Closes #XXX"

# 推送到你的Fork
git push origin feature/my-first-change
```

### 5.2 创建PR

1. 访问你的GitHub Fork页面
2. 点击 "Compare & pull request"
3. 填写PR描述：
   - 标题：简洁说明改动
   - 描述：详细说明改动原因和实现
   - 测试：说明如何验证
4. 提交PR

### 5.3 PR模板示例

```markdown
## 描述
在消息路由流程中添加日志，帮助调试和监控。

## 改动
- `src/routing/router.ts`: 添加路由决策日志
- `src/routing/message-handler.ts`: 添加处理日志

## 测试
- [x] 本地运行验证日志输出
- [x] 所有现有测试通过

## 日志示例
```
[2025-01-15T10:30:00Z] INFO: Routing message: type=text, user=user123
[2025-01-15T10:30:01Z] INFO: Selected handler: TextHandler
```
```

## 6. 常见问题

### 问题1：测试失败

```bash
# 只运行失败的测试
pnpm test -- --reporter=verbose

# 检查具体错误
pnpm test routing/router.test.ts
```

### 问题2：类型错误

```bash
# 运行类型检查
pnpm type-check

# 或
npx tsc --noEmit
```

### 问题3：找不到模块

```bash
# 重新安装依赖
rm -rf node_modules
pnpm install
```

## 今日总结

### 你做到了什么？

1. ✅ **Clone了OpenClaw源码**
2. ✅ **配置了开发环境**
3. ✅ **用模型思维阅读代码**
4. ✅ **修改了一个功能**
5. ✅ **提交了PR**

### 核心收获

```
模型思维 + 动手能力 = 真正的实力
```

### 明天预告

Day 6：《造架构——让AI按你的模型施工》

我们将：
- 设计一个新的功能架构
- 用Prompt 2.0让AI实现
- 集成到OpenClaw

---

## 课后作业

1. **关注你的PR** - 查看维护者的反馈
2. **阅读其他PR** - 学习别人的代码风格
3. **尝试更难的任务** - 挑战中等或困难任务

---

## 资源

- OpenClaw贡献指南：`CONTRIBUTING.md`
- 代码规范：`docs/development/`
- 任务指南：`exercises/task-*.md`